In [ ]:

#pip install transformers datasets torch scikit-learn

In [11]:
from datasets import load_dataset
raw_datasets=load_dataset("imdb")
print(raw_datasets)

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [12]:
from transformers import AutoTokenizer

#load tikenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
  return tokenizer(examples['text'],padding="max_length",truncation=True)

# Apply the function to the dataset
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [13]:
print(tokenized_datasets)


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


Handling Imbalance data

In [14]:
from torch.utils.data import DataLoader,WeightedRandomSampler
import numpy as np

labels=tokenized_datasets['train']['label']
class_weights=1/np.bincount(labels)
sample_weights=[class_weights[label] for label in labels]

# Create sampler
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

# Use in DataLoader
train_dataloader = DataLoader(tokenized_datasets["train"], sampler=sampler, batch_size=16)


Handling noisy data

In [16]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=c6e3196b9d2c3a0d9136500f9d0a569d0ee268cfca125a940403d45330093f39
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [17]:
import re
from langdetect import detect
def clean_text(text):
    # Remove special characters and multiple spaces
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text if detect(text) == "en" else None

raw_datasets = [clean_text(t) for t in raw_datasets]

Using Streaming mode to save memory bottlenecks

In [19]:
from google.colab import drive
drive.mount('/content/drive')

# Check your files
import os
os.listdir('/content/drive/MyDrive')

Mounted at /content/drive


['deeplearning',
 'SE PROJECT REPORT For Kicks (1).docx',
 'SE PROJECT REPORT For Kicks.docx',
 'assessment.gdoc',
 'Untitled document (4).gdoc',
 'API.gdoc',
 'Untitled document (3).gdoc',
 'FYP.docx',
 'Copy of Roadmap.gdoc',
 'Untitled document (2).gdoc',
 'react.gdoc',
 'Colab Notebooks',
 'twitter_training.csv',
 'cleaned_dataset_binary.csv',
 'Untitled document (1).gdoc',
 'Untitled document.gdoc']

In [20]:
dataset=load_dataset("csv",data_files="/content/drive/MyDrive/cleaned_dataset_binary.csv")
# Check the structure
print(dataset)
print(dataset['train'][1])



Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 43374
    })
})
{'text': 'I am coming to the borders and I will kill you all,', 'label': 1}


In [21]:
# See column names
print(dataset['train'].column_names)

['text', 'label']


Streaming mode (streaming=True) loads data one example at a time instead of loading everything into memory at once.

MODEL FINE TUNING

In [22]:
from transformers import AutoModelForSequenceClassification

In [23]:
model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**load_best_model_at_end=True**

When you train a model using the Hugging Face Trainer, the model is saved several times usually once per epoch (depending on save_strategy).

But those saved models are not all equally good. Some epochs perform better than others on the evaluation dataset.

So:

**load_best_model_at_end=True means:**

After training finishes, automatically load the checkpoint that achieved the best evaluation score, not the final epoch.

When you run:

trainer.train()


The Trainer looks at the settings in training_args and does the following:

Saves checkpoints inside /results/
(because you set output_dir="./results")

Evaluates after every epoch (because of evaluation_strategy="epoch")

Saves only one best model
(because of save_total_limit=1)

Loads the best checkpoint at the end (because of load_best_model_at_end=True)

Logs training progress to /logs/(because of logging_dir="./logs")

In [24]:
import transformers
print(transformers.__version__)

4.57.1


In [25]:
from transformers import AutoModelForSequenceClassification

# Load pre-trained BERT for binary classification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2  # Adjust this for multi-class sentiment tasks
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
import torch
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

GPU available: True
GPU name: Tesla T4


In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,  # T4 can handle this for BERT-base
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=1,              # Save space in Colab
    fp16=True,                       # 2x speedup on T4 GPU
    dataloader_num_workers=2,        # Colab has limited CPU cores
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",                # Disable wandb/tensorboard if not needed
)

In [28]:
from transformers import Trainer

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.208500,0.197783
2,0.139100,0.215994
3,0.097800,0.237506


TrainOutput(global_step=2346, training_loss=0.16536663539980784, metrics={'train_runtime': 1170.2021, 'train_samples_per_second': 64.091, 'train_steps_per_second': 2.005, 'total_flos': 9935054899200000.0, 'train_loss': 0.16536663539980784, 'epoch': 3.0})

In [30]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=1)

# Print classification report
print(classification_report(tokenized_datasets["test"]["label"], preds))

              precision    recall  f1-score   support

           0       0.50      0.00      0.00     12500
           1       0.50      1.00      0.67     12500

    accuracy                           0.50     25000
   macro avg       0.50      0.50      0.33     25000
weighted avg       0.50      0.50      0.33     25000



In [33]:
from collections import Counter
print(Counter(tokenized_datasets["train"]["label"]))

Counter({0: 12500, 1: 12500})


In [35]:
# Check if loss decreased during training
for log in trainer.state.log_history:
    if 'loss' in log:
        print(f"Step {log.get('step', 'N/A')}: Loss = {log['loss']:.4f}")

Check if model is actually trained


In [38]:
sample_text=["This movie is absolutely wonderful and amazing!",
    "This movie is terrible and boring."]

from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("distilbert-base-uncased") # Corrected model name
inputs=tokenizer(sample_text,padding=True,truncation=True, return_tensors="pt") # Corrected variable name from sample_texts to sample_text

if torch.cuda.is_available():
  model=model.cuda()
  inputs = {k: v.cuda() for k, v in inputs.items()}

outputs = model(**inputs)
probs = torch.nn.functional.softmax(outputs.logits, dim=1)
print("Predictions:")
print(probs)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Predictions:
tensor([[0.4686, 0.5314],
        [0.4568, 0.5432]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [41]:
# Start fresh with a new model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# Reinitialize the model
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

# Move to GPU
if torch.cuda.is_available():
    model = model.cuda()

# Better training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",           # Evaluate more frequently
    eval_steps=200,                  # Check progress every 200 steps
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Safer batch size
    num_train_epochs=4,              # More epochs
    weight_decay=0.01,
    logging_steps=50,                # See progress
    save_strategy="steps",
    save_steps=400,
    load_best_model_at_end=True,
    fp16=True,
    warmup_steps=500,                # Gradual learning rate warmup
)

# Add evaluation metrics
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

# Create new trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# Train and watch the metrics
trainer.train()




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: selimaharjan33 (selimaharjan33-st-xavier-s-college-nepal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1
200,0.377800,0.311156,0.884600,0.884291
400,0.271900,0.235719,0.910200,0.910051
600,0.266000,0.221832,0.911320,0.911100
800,0.228200,0.211718,0.925520,0.925511
1000,0.266700,0.256774,0.911040,0.910749
1200,0.239200,0.198081,0.928920,0.928876
1400,0.200700,0.213546,0.930720,0.930692
1600,0.170700,0.226835,0.929000,0.928948
1800,0.103900,0.307646,0.926720,0.926639
2000,0.153400,0.250602,0.931560,0.931523


KeyboardInterrupt: 

In [43]:
# Check what checkpoints were saved
import os
import torch # Import torch to use .cuda()

checkpoints = [d for d in os.listdir('./results') if d.startswith('checkpoint')]
print("Saved checkpoints:", sorted(checkpoints))

# Load the best one (around step 1200)
from transformers import AutoModelForSequenceClassification
best_model = AutoModelForSequenceClassification.from_pretrained('./results/checkpoint-1200')

# Explicitly move the loaded model to the GPU if available
if torch.cuda.is_available():
    best_model = best_model.cuda()

# Now evaluate with this model
trainer.model = best_model
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=1)

from sklearn.metrics import classification_report
print(classification_report(tokenized_datasets["test"]["label"], preds))


Saved checkpoints: ['checkpoint-1200', 'checkpoint-1600', 'checkpoint-2000', 'checkpoint-2400', 'checkpoint-2800', 'checkpoint-3200', 'checkpoint-3600', 'checkpoint-400', 'checkpoint-800']


Step,Training Loss,Validation Loss,Accuracy,F1
200,0.377800,0.311156,0.884600,0.884291
400,0.271900,0.235719,0.910200,0.910051
600,0.266000,0.221832,0.911320,0.911100
800,0.228200,0.211718,0.925520,0.925511
1000,0.266700,0.256774,0.911040,0.910749
1200,0.239200,0.198081,0.928920,0.928876
1400,0.200700,0.213546,0.930720,0.930692
1600,0.170700,0.226835,0.929000,0.928948
1800,0.103900,0.307646,0.926720,0.926639
2000,0.153400,0.250602,0.931560,0.931523


              precision    recall  f1-score   support

           0       0.95      0.90      0.93     12500
           1       0.91      0.95      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000



If you want to make prediction faster next time:

best_model = AutoModelForSequenceClassification.from_pretrained(
    './results/checkpoint-1200',
    torch_dtype=torch.float16
).to('cuda')

In [56]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
text="I love this food."
inputs=tokenizer(text,return_tensors="pt")

# Move to GPU
if torch.cuda.is_available():
    inputs = {k: v.cuda() for k, v in inputs.items()}
    model = model.cuda()

label_map={
    0:"Negative",
    1:"Positive"
}

with torch.no_grad(): #only predicting, not training
  pred=(best_model(**inputs).logits.argmax().item())
print("Prediction is:",label_map[pred])

Prediction is: Positive


**inputs expands it into:**

best_model(input_ids=..., attention_mask=...)

**.logits**

The model output contains raw prediction scores for each class.

Example:

logits = [[1.2, -0.3]]


These are NOT probabilities — just scores.

**.argmax()**

Finds the index of the highest score.

Example:

[1.2, -0.3] → highest is 1.2 → index = 0


So your model predicted class 0.

**.item()**

Convert PyTorch tensor → normal Python number.

Example:

tensor(0) → 0 (Python integer)

In [57]:
%cd /content/drive/MyDrive
!git clone https://github.com/seli33/Fine-Tuning-BERT-for-Sentiment-Analysis.git


/content/drive/MyDrive
Cloning into 'Fine-Tuning-BERT-for-Sentiment-Analysis'...


In [58]:
%cd /content/drive/MyDrive/Fine-Tuning-BERT-for-Sentiment-Analysis


/content/drive/MyDrive/Fine-Tuning-BERT-for-Sentiment-Analysis


In [59]:
%cd /content/drive/MyDrive/Fine-Tuning-BERT-for-Sentiment-Analysis
!git status


/content/drive/MyDrive/Fine-Tuning-BERT-for-Sentiment-Analysis
On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)


In [64]:
!ls "/content/drive/MyDrive/Colab Notebooks/"




 basics.ipynb  'Copy of model_training.ipynb'   model_training.ipynb
 BERT.ipynb     FineTuning.ipynb	        Untitled0.ipynb
